In [439]:
%%file ast.lp

#const corange = 10.
constant(1..corange).
%input(1..5).


input(X) :- item(X).
input(X) :- constant(X).


id(0..10).

%%% piece(id,child_id).
3 {piece(I,C):id(I),id(C)}.

%%% All pieces have 0-2 children
:- piece(I,_), not 0 #sum{1,C:piece(I,C)} 2.

%%% All pieces appear at most once as a child
:- id(I), not 0 #sum{1,P:piece(P,I)} 1.
      
%%% Piece can not be its own child
:- piece(P,P).  
    
%%% If a piece appears as a child, it exists
%%% I guess I cant really enforce this without breaking things,
%%% need some way to feed in input.
%%% :- piece(_,P), not piece(P,_).

%%% Pieces can not be children of nodes with a higher id
:- piece(I,C), I>C.
    
piece(I) :- piece(I,_).
piece(I) :- piece(_,I).
leaf(I) :- piece(_,I), not piece(I,_).
    
children(I,N) :- piece(I), N = #count{C:piece(I,C)}.

op(I,xor,V1^V2) :- piece(I,C1), piece(I,C2), C1!=C2, op(C1,T1,V1), op(C2,T2,V2).
op(I,and,V1&V2) :- piece(I,C1), piece(I,C2), C1!=C2, op(C1,T1,V1), op(C2,T2,V2). 
op(I,nop,V1)    :- piece(I,C1), children(I,1), op(C1,T,V1). 
op(I,input,V1)  :- leaf(I), input(V1).  
    
type(xor;and;input).


%%% Every piece has exactly one operation
%%% Really shouldnt be constant, actually the range of values that can be propagated
%%% I guess this would become much more efficient with the bitwise method where everything
%%% is in the range [0,1]


%%% this makes it UNSAT, not sure why
:- piece(I), not 1 #count{T,V:op(I,T,V),type(T),constant(V)} 1.

cnt(I, X) :- piece(I), X = #count{T,V:op(I,T,V),type(T),constant(V)}.
    

    
%&|^~ -+ */ 
%op(P,)

    
%:- idrange(B), not 0 #sum{1,A,C:piece(A,B,C)} 1.
%:- idrange(C), not 0 #sum{1,A,B:piece(A,B,C)} 1.

%cnt(X, A) :- idrange(A), X = #sum{1,B,C:piece(A,B,C)}.
    
#show piece/2.
#show op/3.
#show cnt/2.
#show children/2.

Writing ast.lp


I guess the above would make the basic structure of the AST, but still need some way assign values to the leaf nodes and allow those to be evaluated. No obivous way how to do that.

In [ ]:
!clingo ast.lp 2

clingo version 5.2.0
Reading from ast.lp
ast.lp:7:13-20: info: atom does not occur in any rule head:
  item(X)



# Everything below is garbage

In [ ]:
!clingo items.lp perfect.lp

clingo version 5.2.0
Reading from items.lp ...


In [187]:
%%file genhash.lp

#const corange = 10.
constant(1..corange).
input(1..5).


idrange(0..10).



%%% piece(id,child_id_1,child_id_2).
2 {piece(A,B,C):idrange(A),idrange(B),idrange(C)}.

%%% At most one piece per id
:- idrange(A), not 0 #sum{1,B,C:piece(A,B,C)} 1.
        
%%% Piece appears at most once as a child
:- idrange(P), not 0 #sum{1,A,C,left:piece(A,P,C); 1,A,B,right:piece(A,B,P)} 1.
    
%%% Piece can not be its own child
:- idrange(P), piece(P,P,_).
:- idrange(P), piece(P,_,P).
    
%%% If a piece appears as a child, it exists
%TODO
    

    
%:- idrange(B), not 0 #sum{1,A,C:piece(A,B,C)} 1.
%:- idrange(C), not 0 #sum{1,A,B:piece(A,B,C)} 1.

%cnt(X, A) :- idrange(A), X = #sum{1,B,C:piece(A,B,C)}.
    
    

Overwriting genhash.lp


In [21]:
%%file perfect.lp

%%% Attempts to perfectly hash keys given as item(number)

#const corange = 10.
constant(1..corange).
    
%%%operation(type, item, in1, in2, output)
operation(xor, I, A, B, A^B) :- item(I), output(I,A), output(I,B).       
operation(and, I, A, B, A&B) :- item(I), output(I,A), output(I,B). 
operation(or , I, A, B, A?B) :- item(I), output(I,A), output(I,B). 
    

hash(I,X) :- item(I), input(I,V1), input(I,V2), operation(T,I,V1,V2,X).
    
    
output(I,I) :- item(I).
output(I,X) :- item(I), constant(X).
output(I,X) :- item(I), output(I,V1), output(I,V2), operation(T,I,V1,V2,X).
    
    
:- item(A), item(B), A != B, hash(A,X), hash(B,X). 
    
    
    
%%% input constant or item
input(I) :- item(I).
input(I) :- constant(I).
    
    
%:- item(A), item(B), operation(T,ID,) 
    
    
%%output(I,X) :- input(I), operation(T, )
    




Overwriting perfect.lp


In [12]:
%%file minimal.lp

%%% Minimize the range of hashed values
maxhash(X) :- coef(A), mod(M), exp(E), X = #max{ @axbmodc(A,I,E,M):item(I) }.
minhash(X) :- coef(A), mod(M), exp(E), X = #min{ @axbmodc(A,I,E,M):item(I) }.
diff(X) :- maxhash(A), minhash(B), X = A-B.
#minimize{ X:diff(X) }.

Overwriting minimal.lp


In [45]:
%%file ordered.lp

%%% Reject if items are out of order when hashed
:- item(I1), item(I2), I1 < I2, coef(A), mod(M),  A*I1\M > A*I2\M.
:- #true.

Overwriting ordered.lp


In [186]:
!clingo genhash.lp

clingo version 5.2.0
Reading from genhash.lp
Solving...
Answer: 1
constant(1) constant(2) constant(3) constant(4) constant(5) constant(6) constant(7) constant(8) constant(9) constant(10) input(1) input(2) input(3) input(4) input(5) idrange(0) idrange(1) idrange(2) idrange(3) idrange(4) idrange(5) idrange(6) idrange(7) idrange(8) idrange(9) idrange(10) piece(0,7,10) piece(7,0,1)
SATISFIABLE

Models       : 1+
Calls        : 1
Time         : 0.437s (Solving: 0.01s 1st Model: 0.01s Unsat: 0.00s)
CPU Time     : 0.430s


In [38]:
%%file process.py

import json
import sys
import re


result = json.load(sys.stdin)
atoms = result['Call'][0]['Witnesses'][0]['Value']

items = []
modulus = 0
coef = 0
exponent = 0

for a in atoms:
    match = re.match('item\(([0-9]*)\)', a)
    if match:
        items += [int(match.group(1))]
    
    match = re.match('mod\(([0-9]*)\)', a)
    if match:
        modulus = int(match.group(1))

    match = re.match('exp\(([0-9]*)\)', a)
    if match:
        exponent = int(match.group(1))
        
    match = re.match('coef\(([0-9]*)\)', a)
    if match:
        coef = int(match.group(1))

        
table = [(item*coef**exponent%modulus, item) for item in items]
table.sort()

print "Hash function: %dx^%d mod %d" %(coef,exponent,modulus)

print "%7s %7s" % ("hash", "item")
for item in table:
    print "%7d %7d" % item

Overwriting process.py


In [69]:
!clingo items.lp perfect.lp minimal.lp -t 8 --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 49x^1 mod 36
   hash    item
      3       3
      4     100
      6       6
      7      67
      9       9
     10   34234
     11      23
     13       1
     15     123
     16       4
     19       7
     22      10
     26       2
     29       5
     31      55
     32       8
     35   23123


In [12]:
%%file gen.py
import random

seen = {}
i = 0
while i < 5:
    x = random.randint(10,100)
    if x not in seen:
        print "item(%d)." % (x)
        seen[x] = 1
        i+=1

Overwriting gen.py


In [154]:
!python2 gen.py > gen.lp; clingo gen.lp perfect.lp minimal.lp ordered.lp quiet.lp --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 55x mod 57
   hash    item
     25      16
     38      38
     44     149
     45     291
     49     631


In [67]:
!python2 gen.py > gen.lp; clingo gen.lp perfect.lp minimal.lp -t 8 --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 24x^1 mod 11
   hash    item
      3      62
      4      90
      5      96
      6      91
      8      81
